In [1]:
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras_nlp
import keras
import tensorflow as tf
from keras import layers
from keras.layers import TextVectorization
from dataclasses import dataclass
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint
from nltk.tokenize import WhitespaceTokenizer
import re 
import sys 
from tqdm import tqdm
import time

2024-05-14 20:12:54.692051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 20:12:55.656252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [2]:
import importlib
import model 
import config
import dataset
from config import get_config, set_config_ratio
from dataset import get_dataset_class
from model import compile_model, get_optimizer
importlib.reload(model)
importlib.reload(dataset) 
importlib.reload(config)

<module 'config' from '/home/jovyan/config.py'>

## Setup WandB

In [5]:
import wandb 

In [6]:
wandb.login()

wandb: Currently logged in as: arnavkartikeya. Use `wandb login --relogin` to force relogin


True

In [3]:
config = get_config() 
dataset = get_dataset_class(config) 

In [4]:
mlm_ds = dataset.generate_dataset().shuffle(1000).batch(config.BATCH_SIZE)

2024-05-14 20:12:59.690193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22400 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:89:00.0, compute capability: 8.6


starting generating masks...
starting generating mask for 0/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 1000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 2000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 3000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 4000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 5000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 6000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 7000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 8000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 9000/50000 sentence...
finished generating mask for 1 sentence
starting generating mask for 10000/50000 sentence...
f

### Cell below test serialization

In [8]:
my_model = compile_model(config)
out = next(iter(mlm_ds.take(1)))
# x, y, mask, padding_mask, sample_weights = out
# before_save_pred = my_model((x, mask, padding_mask))
# my_model.save('./my_tf_model')
# saved_model = keras.models.load_model('./my_tf_model', compile=False) 
# # saved_model.summary()
# saved_model.compile(optimizer='sgd', metrics=["accuracy"]) #add parameter run_eagerly=True if this shit doesnt work 
# saved_model((x, mask, padding_mask))

In [8]:
def predict(model, dataset, text, k, output_attention):
    #k = get the top k answers as an array
    encoded_text = dataset.encode(text) 
    mask_index = np.where(encoded_text == 29999)[0][0] 
    mask, padding_mask = dataset.generate_mask(encoded_text)
    # print(f'mask shape: {mask.shape}') 
    # print(f'padding mask: {padding_mask.shape}')
    encoded_text = np.expand_dims(encoded_text, axis=0)
    mask = np.expand_dims(mask, axis=0)
    padding_mask = np.expand_dims(padding_mask, axis=0)

    # print(mask_index)

    encoded_text_tensor = tf.convert_to_tensor(encoded_text, dtype=tf.int32)
    mask_tensor = tf.convert_to_tensor(mask, dtype=tf.bool)
    padding_mask_tensor = tf.convert_to_tensor(padding_mask, dtype=tf.bool)
    

    data = (encoded_text_tensor, mask_tensor, padding_mask_tensor, sample_weights) 
    if output_attention: 
        prediction, attention = model(data)
        prediction = dataset.decode(np.argsort(prediction[0][mask_index])[::-1][:k])
        prediction = prediction.split() 
        return prediction, attention
    prediction, _ = model(data)
    prediction = dataset.decode(np.argsort(prediction[0][mask_index])[::-1][:k])
    prediction = prediction.split() 
    return prediction

In [9]:
def measure_acc(logits, x_input, y_input, verbose=0):
    '''
    logits: Tensor, shape: (batch, 256, 30000)
    x_input: Tensor, shape: (batch, 256) 
    y_input: Tensor, shape: (batch, 256) 
    ''' 
    logs = logits.numpy()
    logs = np.argmax(logs, axis=-1).flatten() #shape: (batchx256) 

    # x = x_input.numpy().flatten() 
    y = y_input.numpy().flatten() 
    
    pred_diff = np.count_nonzero(np.abs(logs-y)) 

    return (pred_diff)
    

In [10]:
attentions = [] 
config.NUM_LAYERS=2
def train(epochs=2, config=config, mlm_ds=mlm_ds, step_size_ratio=1, dataset=dataset, log_to_wandb=False):
    global attentions
    ratio_ranges = np.arange(0, 1+step_size_ratio, step_size_ratio) 

    text = "there was this great [mask] at the theatre today, it was quite long but also really fun to watch with friends"
    k = 5
    output_attention = True 

    model_to_return = None
    
    for ratio in ratio_ranges:
        if ratio >= 1: 
            break 
        config.RATIO = ratio
        print(f"Training model for ratio = {config.RATIO}") 
        print("building model with custom config") 
        model = compile_model(config)
        print("finished building model") 
        optimizer = get_optimizer() 
        loss_fn = keras.losses.SparseCategoricalCrossentropy(reduction="none")
        loss_tracker = keras.metrics.Mean(name="loss")
        epoch_count = 0
        print("started training")
        run = None 
        if(log_to_wandb): 
            run = wandb.init(
                # Set the project where this run will be logged
                project="encoder-mlm-real",
                # Track hyperparameters and run metadata
                config={
                    "ratio": ratio,
                    "epochs": epochs,
                    "lr": config.LR
                },
            )
            run.name=f"Ratio: {ratio}, Epochs: {epochs}" 
        for epoch in range(epochs): 
            for step, (x_input, y_input, mask, padding_mask, sample_weights) in enumerate(mlm_ds): 
                start_time = time.time()
                with tf.GradientTape() as tape: 
                    # print(f"mask shape: {mask.shape}")
                    logits, _ = model((x_input, mask, padding_mask)) #do not output attention 
                    loss_value = loss_fn(y_input, logits, sample_weight=sample_weights)
                    
                grads = tape.gradient(loss_value, model.trainable_weights)
                
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
                loss_tracker.update_state(loss_value, sample_weight=sample_weights)

                # acc = measure_acc(logits, x_input, y_input, verbose=1) 
                if(log_to_wandb):
                    wandb.log({"loss": loss_tracker.result(), "acc": acc})

                if step%100 != 0:
                    print(f"\rEpoch: {epoch}, Step {step + 1}, Loss: {loss_tracker.result().numpy():.4f}, Time: {time.time()-start_time:.3f} sec", end='', flush=True)
                
                if step % 100 == 0:
                    print("GETTING ACCURACY") 
                    print("------------------------") 
                    avg = 0 
                    preds = logits.numpy().argmax(axis=-1) 
                    for i in range(16):
                        indices = np.where(x_input.numpy()[i] == 29999)[0]
                        if i == 15:                            
                            print("predictions")
                            print(dataset.decode(y_input.numpy()[i][indices]))
                            print("outputs") 
                            print(dataset.decode(preds[i][indices]))
                        print(f"accuracy: {1-(np.count_nonzero(y_input.numpy()[i][indices] - preds[i][indices]))/y_input.numpy()[i][indices].shape[0]}")
                        print("------------------------") 
                    # results = model.evaluate(mlm_ds
                    # print("Training loss (for one batch) at step %d: %.4f"% (step, float(loss_value))
                    # )
                    # print("Seen so far: %s samples" % ((step + 1) * batch_size))
            epoch_count += 1
            #save the model 
        model.save(f"saved_models/model_ratio_{config.RATIO}")
        if(log_to_wandb):
            wandb.finish()
        
        # # return model #adding this for now 
        # print("--------------EXAMPLE PREDICTION----------------")
        # prediction, attention = predict(model, dataset, text, k, output_attention)
        # attentions.append(attention)
        # print(prediction)
        # print("--------------END OF PREDICTION----------------")
        # model_to_return = model 
    return None 


In [11]:
model = train(epochs=20, step_size_ratio=0.5)

Training model for ratio = 0.0
building model with custom config
finished building model
started training


2024-05-14 20:24:33.681854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-05-14 20:24:33.746381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-05-14 20:24:34.109209: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5624cc4b1810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-14 20:24:34.109249: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-05-14 20:24:34.116192: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-14 20:24:34.342472: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

GETTING ACCURACY
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
------------------------
predictions
goer my short years surprised this did sometime i this it
outputs
thoughtless steiner drugs aided fisherman botch infiltrating wheaton aided infiltrating defends
accuracy: 0.0
------------------------
Epoch: 0, Step 100, Loss: 10.2593, Time: 0.451 secGETTING ACCURACY
------------------------
accuracy: 0.0
------------------------
accuracy: 0.0
---------------

KeyboardInterrupt: 

In [5]:
saved_model_0 = keras.models.load_model('saved_models/model_ratio_0.0', compile=False)
saved_model_8 = keras.models.load_model('saved_models/model_ratio_0.5', compile=False) 
optim = get_optimizer()
saved_model_0.compile(optimizer=optim, metrics=['accuracy'])
saved_model_8.compile(optimizer=optim, metrics=['accuracy']) 

In [6]:
y = "there was this great film at the theatre today, it was really long but also very fun to watch with friends"
text = "there was this great film at the theatre today, it was really long but also [mask] fun to watch with friends"
#k = get the top k answers as an array
encoded_text = dataset.encode(text) 
mask_index = np.where(encoded_text == 29999)[0][0] 
mask, padding_mask = dataset.generate_mask(encoded_text)
# print(f'mask shape: {mask.shape}') 
# print(f'padding mask: {padding_mask.shape}')
encoded_text = np.expand_dims(encoded_text, axis=0)
mask = np.expand_dims(mask, axis=0)
padding_mask = np.expand_dims(padding_mask, axis=0)

encoded_text_tensor = tf.convert_to_tensor(encoded_text, dtype=tf.int32)
mask_tensor = tf.convert_to_tensor(mask, dtype=tf.bool)
padding_mask_tensor = tf.convert_to_tensor(padding_mask, dtype=tf.bool)


data = (encoded_text_tensor, mask_tensor, padding_mask_tensor) 

In [7]:
out_0, attention_0 = saved_model_0(data) 
out_8, attention_8 = saved_model_8(data)

2024-05-14 03:01:21.355890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-05-14 03:01:21.363842: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


In [6]:
x = None 
y = None 
weights = None 
out = None 
for (x_data_tensor, y_data_tensor, syntactic_mask_tensor, padding_mask_tensor, sample_weights_tensor) in mlm_ds:
    y = y_data_tensor 
    x = x_data_tensor
    input = (x_data_tensor, syntactic_mask_tensor, padding_mask_tensor)
    break 
out = saved_model_0(input)
preds = np.argmax(out[0].numpy(), axis=-1)
for i in range(16):
    indices = np.where(x.numpy()[i] == 29999)[0]
    print("predictions")
    print(dataset.decode(y.numpy()[i][indices]))
    print("outputs") 
    print(dataset.decode(preds[i][indices]))
    print(f"accuracy: {1-(np.count_nonzero(y.numpy()[i][indices] - preds[i][indices]))/y.numpy()[i][indices].shape[0]}")
    print("------------------------") 

NameError: name 'saved_model_0' is not defined

In [8]:
x = None 
y = None 
for (x_data_tensor, y_data_tensor, syntactic_mask_tensor, padding_mask_tensor, sample_weights_tensor) in mlm_ds:
    y = y_data_tensor 
    x = x_data_tensor
    input = (x_data_tensor, syntactic_mask_tensor, padding_mask_tensor)
    break 
print(f"x data tensor: {x[0]}") 
print("-------------------------")
print(f"y data tensor: {y[0]}") 
print("-------------------------")
print(f"x data tensor: {sample_weights_tensor[0]}") 
print("-------------------------")
print(f"number of masked tokens: {np.where(x[0] == 29999)[0].shape[0]}") 

x data tensor: [ 2585 10289     7 10491  1024   184   533    36    15  9444    35     4
     1  1476    45     4 29999     5  1028 29999     2   395    36 10097
    41    82    11 29999     7    21   289    36  2112   229    42    41
   627 29999  1717     8     2   755  3540    32  1913    36   309     8
 21743     6   160   154    17 29999  5120    82 29999    36   192    11
 29999    57  1295     6   245     9    13   860 29999     2  1037   172
    97    60    25   265   479    39   283 29999     5   759    82     2
   100     3     2   172   515   150   450     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0

In [86]:
from bertviz import model_view

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [95]:
viz_attention_0 = attention_0 
viz_attention_0[0] = attention_0[0][:, :, :21, :21].numpy()
viz_attention_0[1] = attention_0[1][:, :, :21, :21].numpy()

viz_attention_8 = attention_8  
viz_attention_8[0] = attention_8[0][:, :, :21, :21].numpy()
viz_attention_8[1] = attention_8[1][:, :, :21, :21].numpy()

In [97]:
import torch 
tf_0= viz_attention_0
tf_8 = viz_attention_0
tf_0[0] = torch.from_numpy(viz_attention_0[0]) 
tf_0[1] = torch.from_numpy(viz_attention_0[1]) 
tf_8[0] = torch.from_numpy(viz_attention_8[0]) 
tf_8[1] = torch.from_numpy(viz_attention_8[1]) 

In [102]:
html_model_view_0 = model_view(tf_0, text.split(), html_action='return')
with open("attention_viz/model_view_0.html", 'w') as file:
    file.write(html_model_view_0.data)

html_model_view_8 = model_view(tf_8, text.split(), html_action='return')
with open("attention_viz/model_view_8.html", 'w') as file:
    file.write(html_model_view_8.data)

In [61]:
dataset.decode(np.argmax(out[0][0].numpy(), axis=-1))

'there was this great film at the as today it was [UNK] long but also a fun to watch with friends'

In [22]:
dataset.encode("reigning")

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [45]:
for key in dataset.id2token:
    if dataset.id2token[key] == "[mask]": 
        print(key) 

In [23]:
print(dataset.decode([29999]))

[mask]


In [32]:
preds = prediction[0]
print(np.max(preds, axis=-1))
print(np.argmax(preds, axis=-1))

[0.99458414 0.9898177  0.9917646  0.99501306 0.99558616 0.99439055
 0.9988035  0.9817747  0.97407585 0.99605536 0.99747664 0.9984666
 0.9944267  0.99824774 0.9892838  0.04622511 0.9959009  0.9979032
 0.9920459  0.9979886  0.9969368  0.9999248  0.9999249  0.99992
 0.9999112  0.9999014  0.9998957  0.99989533 0.9998995  0.9999124
 0.99992716 0.99993575 0.9999378  0.9999378  0.99994123 0.9999453
 0.99994934 0.9999529  0.99995625 0.9999573  0.99995816 0.9999583
 0.99995756 0.99995637 0.99995625 0.9999554  0.9999536  0.9999505
 0.99994874 0.9999511  0.99995506 0.99995637 0.9999552  0.99995244
 0.99994886 0.99994385 0.9999398  0.99993885 0.99994147 0.9999466
 0.9999503  0.99995375 0.99995565 0.99995613 0.99995494 0.9999552
 0.99995637 0.9999572  0.99995625 0.9999541  0.99995077 0.9999472
 0.99994504 0.9999455  0.9999509  0.9999516  0.9999492  0.99994934
 0.99995255 0.999954   0.9999529  0.99995124 0.999946   0.9999386
 0.9999366  0.9999374  0.99994206 0.9999478  0.99995184 0.9999523
 0.999951

In [29]:
dataset.decode(dataset.encode("this is a [mask]"))

'this is a reigning'

In [13]:
preds = prediction[0]
preds = np.argmax(preds, axis=-1)
real_text = y.split()
for correct, pred in zip(real_text, preds): 
    if pred == 0:
        break 
    else: 
        print("========================")
        print(f"correct: {correct}")
        print(f"prediction: {dataset.decode([pred])}")
        print("========================")

correct: there
prediction: there
correct: was
prediction: was
correct: this
prediction: this
correct: great
prediction: great
correct: film
prediction: film
correct: at
prediction: at
correct: the
prediction: the
correct: theatre
prediction: theatre
correct: today,
prediction: today
correct: it
prediction: it
correct: was
prediction: was
correct: super
prediction: film
correct: long
prediction: long
correct: but
prediction: but
correct: also
prediction: also
correct: really
prediction: it
correct: fun
prediction: fun
correct: to
prediction: to
correct: watch
prediction: watch
correct: with
prediction: with
correct: friends
prediction: friends


## Attention and Other Shit

In [95]:
print(attentions[0][0][0][:21, :21][0][0].numpy())

[0.03773842 0.01787496 0.01448925 0.01764323 0.02313525 0.0369661
 0.05562109 0.08916617 0.08994694 0.09715331 0.07790561 0.08223583
 0.06563642 0.04593078 0.03129031 0.04033205 0.03472701 0.03084017
 0.02977602 0.03660338 0.04498763]


In [24]:
from bertviz import head_view, model_view

In [25]:
import numpy as np

# Assuming attentions[0][1].numpy() is 2D array of size larger than (9, 9)
test = attentions[0][1].numpy()

print(test[:,:,:21,:21].shape)  # Extract top left corner of size 9x9


(1, 8, 21, 21)


In [82]:
import torch 
temp_attention = attentions 
temp_attention[0][0] = torch.tensor(attentions[0][0].numpy()[:,:,:21,:21])
# temp_attention[0][1] = torch.tensor(attentions[0][1].numpy()[:,:,:21,:21])
temp_attention[1][0] = torch.tensor(attentions[1][0].numpy()[:,:,:21,:21])
# temp_attention[1][1] = torch.tensor(attentions[1][1].numpy()[:,:,:21,:21])

In [62]:
import torch 
type(torch.tensor(attentions[0][0].numpy()))

torch.Tensor

In [84]:
tokens = "there was this great [mask] at the theatre today, it was quite long but also really fun to watch with friends".split()
html_model_view = model_view(temp_attention[0], tokens, html_action="return")
with open("./model_view.html", 'w') as file:
    file.write(html_model_view.data)

In [36]:
len(temp_attention[0])

2

In [85]:
html_model_view = model_view(temp_attention[1], tokens, html_action="return")
with open("./model_view_syntactic.html", 'w') as file:
    file.write(html_model_view.data)

In [104]:
count = 0 
for step, (x_input, y_input, syntactic_mask, padding_mask) in enumerate(mlm_ds): 
    logits = model((x_input, mask, padding_mask))
    print(logits.shape)
    if count == 0:
        break

(32, 256, 30000)


In [131]:
print(x_input[1].numpy())
print(np.where(x_input[1].numpy() == 29999))

[    9  2658    11     7 29999   709    19  4279    15     2  4211    20
   184   480    15    73   944     1     7     9     2   709    23    37
 23288 29999    11   237     5   146 29999     2   631  1448     7    11
    80     9    62   262   124    14 28497 29999     1     7   304    59
  3576 29999   572  4294     6 29999 15859     5 23565 11081  6344     3
  5283   284     8     4   724   822    18    59 29999 29999  1140 29999
     2   294   386 29999    41  3750  8584     1  1884     7   237    76
  5390    25   222     2   159     5  3967     2  1276  1837  6993  3575
     8    11   411     5     2   145 12558     4  7307  1808     3    28
     5     2 29999  4035 29999 16472     5    11   822 14750  1105 29999
    65  2383     6   169 29999   702   206 29999    84 29999     2 29999
 29999    12    31    80  2575    66   339    59   179    73 14148    12
 19395     6  1639 29999 29999     2    65     5    41  1711  3108     7
    37  2971    21    60 29999    59 13897 13914   

In [132]:
dataset.decode(np.array([x_input[].numpy()[4]]))

'reigning'

In [137]:
print(dataset.decode(x_input[0].numpy()))

anyone who has a remote reigning in science fiction should start at the basics everyone says star wars and star trek are the best mayan fiction films reigning begin at which is fine but the truth is the terminator and this movie reigning green are far better choices reigning those reigning reigning is probably science reigning s reigning kept secret it remains one of the biggest yet most forgotten reigning but the impact of its setting is tilted reigning a reality with each passing day charlton reigning [UNK] his role yet it works edward g robinson in his final role makes the most out of it in soylent reigning more than anyone else and reigning final reigning are touching it is manhattan in 2022 the world is [UNK] and reigning is an unbelievable fortune a cosmic reigning of strawberry jam costs 150 a big executive for the reigning company is murdered reigning police detective thorn is on the case the secret of soylent green reigning not a reigning if you do research reigning the movie 

In [138]:
print(dataset.decode(y_input[0].numpy()))

anyone who has a remote interest in science fiction should start at the basics everyone says star wars and star trek are the best science fiction films to begin at which is fine but the truth is the terminator and this movie soylent green are far better choices than those series soylent is probably science fiction s best kept secret it remains one of the biggest yet most forgotten films but the impact of its setting is becoming more a reality with each passing day charlton heston [UNK] his role yet it works edward g robinson in his final role makes the most out of it in soylent green more than anyone else and his final scenes are touching it is manhattan in 2022 the world is [UNK] and food is an unbelievable fortune a small jar of strawberry jam costs 150 a big executive for the soylent company is murdered and police detective thorn is on the case the secret of soylent green is not a mystery if you do research on the movie soylent is enjoyable to watch but the whole screenplay is a jok

In [136]:
dataset.decode(np.argmax(logits[0], axis=-1)) 

'anyone who has a remote movie in science fiction should start at the basics everyone says star wars and star trek are the best mayan fiction films it begin at which is fine but the truth is the terminator and this movie it green are far better choices and those and it is probably science it s it kept secret it remains one of the biggest yet most forgotten that but the impact of its setting is is to a reality with each passing day charlton and [UNK] his role yet it works edward g robinson in his final role makes the most out of it in soylent and more than anyone else and and final and are touching it is manhattan in 2022'

In [ ]:
indices = np.where(x_input.numpy() == 29999)
indices

In [151]:
indices[0].dtype

dtype('int64')

In [177]:
dataset.decode(y_input[2].numpy()[indices])

'a viewed the cover vision a interesting than you your is you i sympathy character fact downright unlikable as a forced trust little which hallucination source cover images film never event oh'

In [178]:
dataset.decode(np.argmax(logits[2].numpy()[indices], axis=-1)) 

'a viewed the cover and a interesting than you it is you i and to to downright unlikable as a forced trust little which hallucination source cover images film never event oh'

In [168]:
np.argmax(logits[0].numpy()[indices], axis=-1)

array([17,  9,  9,  3,  3,  9,  9,  9, 12,  6,  3,  3,  3,  3,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [77]:
def measure_acc(logits, x_input, y_input, verbose=0):
    '''
    logits: Tensor, shape: (batch, 256, 30000)
    x_input: Tensor, shape: (batch, 256) 
    y_input: Tensor, shape: (batch, 256) 

    ''' 

    total_correct = 0 
    total = 0 

    print(f"x_input shape: {x_input.shape}")
    
    for i in range(x_input.shape[0]): 
        x = x_input[i].numpy() #shape(1, 256) 
        y = y_input[i].numpy() #shape (1, 256) 
        indices = np.where(x_input[i] == 29999)
        logs = logits[i].numpy() 

        logs = logs[indices] 
        logs = np.argsort(logs)[::-1] #(18, 30000)
        logs = logs[:,:5] # (18, 5) 

        correct = y[indices] #(18,) 

        print(f"logs shape: {logs.shape}")
        print(f"correct shape: {correct.shape}")

        for j in range(logs.shape[0]): 
            if correct[j] in logs[j]: 
                total_correct += 1 
            total += 1 

        if verbose == 1: 
            print(correct)
            print(f"correct: {dataset.decode(correct)}")
            pred = [] 
            for i in logs: 
                pred.append(dataset.decode(i)) 
            print(f"preds: {pred}")
        return total_correct/total 
    
    # print(f"x_input shape: {x_input.shape}")
    # indices = np.where(x_input == 29999) #type: tuple, shape: (32, num_masks) 

    # print(f"indices shape: {len(indices)}")
    # print(f"indices real shape: {indices[0].shape}")
    
    # logs = logits.numpy() 
    # logits_top_k = np.argsort(logs, axis=-1)[::-1] #shape (32, 256, 30000)
    # logits_top_k = logits_top_k[:,:,:5] #shape (32, 256, 5) 

    # print(f"logits_top_k shape: {logits_top_k.shape}")

    # y = y_input.numpy() 

    # i = 0 
    # num_correct = 0 
    # num_total = 0 
    # for indx in indices: 
    #     logits_top_k_preds = logits_top_k[i, indx, :] #shape (1, num_masks, 5)  
    #     print(f"logits_top_k_preds shape: {logits_top_k_preds.shape}")
    #     mask_answers = y[i, indx] #shape (1, num_masks)

    #     print(f"mask_answers shape: {mask_answers.shape}")

    #     for i in range(logits_top_k.shape[1]):
    #         if mask_answers[i] in logits_top_k[0, i, :]:
    #             num_correct += 1
    #     num_total += logits_top_k.shape[1] 
    # return num_correct/num_total
        
    

    # total = [0, 0] #(total correct, total guessed) 
    
    # for i in range(batch): 
    #     indices = np.where(x_input[i] == 29999)

    #     logits_argsort = np.argsort(logits[i], axis=-1)[::-1] 
    #     preds = logits_argsort[i,i,:5] #shape (32, 256, 5) 

    #     correct_words = y_input[i].numpy()[indices]

    #     count = 0 
    #     for i in range(min(len(predicted_words), len(correct_words))): 
    #         if correct_words[i] in preds:
    #             total[i] += 1
    #         total[i] += 1 
    return total[0]/total[1]      
        
    

In [96]:
print(measure_acc(logits, x_input, y_input))

x_input shape: (32, 256)
logs shape: (26, 5)
correct shape: (26,)
[  895    48     3  1809   121  3672  1002  9443    15 11424 19187   136
    50   365   784  8174   780     5   119     6   435    23  9617 16414
    59    82]
correct: write what and page does attached party habits with maniacal intrinsic such when start subject avoids elements of life to 3 are therein sharma she into
preds: ['incident melodrama unconventional pertaining endear', 'abject kanes fortitude snake translating', 'alumni sexless goines outspoken commitment', 'slums dread alumni goines prof', 'pales chaplin populate furious underlining', 'elegant sentinel conroy sooooo furious', 'improvisational many detective furious tequila', 'rudy skipper begin trainer hoss', 'alumni garbo 69 performers slovenian', 'alumni 69 garbo rudy moronic', 'unconsciousness sondheims slave wisely virgil', 'enthusiastic enemys driveway cbc flirtatious', 'wisely virgil personifies experimented literal', 'wisely seminal keerthana robe sof

In [14]:
x_input = None 
y_input = None 
mask = None 
padding_mask = None
for step, (x_input, y_input, mask, padding_mask) in enumerate(mlm_ds): 
    x_input = x_input 
    y_input = y_input 
    mask = mask 
    padding_mask = padding_mask 
    break 

In [44]:
logits = model((x_input, mask, padding_mask)) 

In [45]:
type(logits)

tensorflow.python.framework.ops.EagerTensor

In [31]:
logits.numpy()[0,0,logits_argsort[0,0,:-5]]

array([2.3624720e-05, 2.3650602e-05, 2.4203482e-05, ..., 4.5484216e-05,
       4.5081844e-05, 4.4933997e-05], dtype=float32)

(32, 256, 30000)

In [23]:
logits_argsort[0][0][:5]

array([12650,  4478, 17189, 29892, 13777])

In [30]:
logits_argsort = np.argsort(logits, axis=-1)[::-1] 
preds = logits_argsort[0,0,:5]

array([[[2.3624720e-05, 2.3650602e-05, 2.4203482e-05, 2.4268829e-05,
         2.4722051e-05],
        [2.4384562e-05, 2.3588826e-05, 2.4317258e-05, 2.4596175e-05,
         2.6424554e-05],
        [2.5367803e-05, 2.4165518e-05, 2.4917123e-05, 2.5199330e-05,
         2.8532308e-05],
        ...,
        [2.9357727e-05, 3.0069335e-05, 2.9820629e-05, 3.0462938e-05,
         2.6266240e-05],
        [2.9973031e-05, 3.0018720e-05, 3.0581850e-05, 2.9653955e-05,
         2.6903224e-05],
        [3.0156471e-05, 3.0119894e-05, 3.1214320e-05, 2.8417953e-05,
         2.7636048e-05]],

       [[2.3500072e-05, 2.3662180e-05, 2.4121982e-05, 2.4358731e-05,
         2.4852736e-05],
        [2.4267056e-05, 2.3901115e-05, 2.4372990e-05, 2.4697101e-05,
         2.6273945e-05],
        [2.5041390e-05, 2.4129708e-05, 2.4778832e-05, 2.4841740e-05,
         2.8371784e-05],
        ...,
        [2.9363466e-05, 3.0075573e-05, 2.9824765e-05, 3.0461506e-05,
         2.6261863e-05],
        [2.9978637e-05, 3.002478

In [24]:
print(logits[logits_argsort[0,0,:5]])

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([12650,  4478, 17189, 29892, 13777])

In [13]:
model = compile_model(config)


In [32]:
logits = np.array([0, 1, 2, 3, 4 ,5 ]) 
np.argsort(logits)[::-1][:5]

array([5, 4, 3, 2, 1])

In [86]:
correct = [  28, 40 ,   5  , 65 ,  24, 1504 ,1646  , 59 ,7182 ,  41 , 123,  336 , 790  ,379, 152, 1543 , 145 ,  89]
dataset.decode(correct)

None


TypeError: 'NoneType' object is not subscriptable